In [0]:
%run 
"/Workspace/Users/pulianand@yahoo.com/dbtraining/CES DBX Training/Day 5 - Medallion _Multihop_Architecture/includes"

In [0]:
dbutils.widgets.text("catalog","dev_catalog") 
dbutils.widgets.text("schema","default") 
dbutils.widgets.text("tablename","tbl") 

 

catalog=dbutils.widgets.get("catalog") 
schema=dbutils.widgets.get("schema") 
tablename=dbutils.widgets.get("tablename") 

In [0]:
df = spark.read.table(f"{catalog}.{schema}.{tablename}")

df_final = df.dropna()
df_final.count()

In [0]:
df_final = df.drop_duplicates().dropna().drop("ingestion_date")
df_final.count()

display(df_final)

In [0]:
df_final.write.mode("overwrite").saveAsTable("dev_catalog.silver.sales_cleaned1") 

In [0]:
%sql
create or replace table dev_catalog.silver.produts_scd(product_id int, product_name string, product_category string, product_price double)

MERGE INTO dev_catalog.silver.products_scd AS t
USING dev_catalog.bronze.product AS s
ON t.product_id = s.product_id
WHEN MATCHED THEN
  UPDATE SET
    t.product_name = s.product_name,
    t.product_category = s.product_category,
    t.product_price = s.product_price
WHEN NOT MATCHED THEN
  INSERT (product_id, product_name, product_category, product_price)
  VALUES (s.product_id, s.product_name, s.product_category, s.product_price)

WITH deduplicated_source AS ( 
SELECT 
s.*, 
ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY seqNum DESC) as row_num 
FROM dev_catalog.bronze.product s 
) 
SELECT *
FROM deduplicated_source
WHERE row_num = 1



CREATE OR REPLACE TABLE dev_catalog.silver.products_scd (product_id int, product_name string, product_category string, product_price double) 

 

WITH deduplicated_source AS ( 
SELECT 
s.*, 
ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY seqNum DESC) as row_num 
FROM dev_catalog.bronze.product s 
) 
MERGE INTO dev_catalog.silver.products_scd AS t
USING dev_catalog.bronze.product AS s
ON t.product_id = s.product_id
WHEN MATCHED THEN
  UPDATE SET
    t.product_name = s.product_name,
    t.product_category = s.product_category,
    t.product_price = s.product_price
WHEN NOT MATCHED THEN
  INSERT (product_id, product_name, product_category, product_price)
  VALUES (s.product_id, s.product_name, s.product_category, s.product_price)